In [1]:
import gensim

embeddings = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)

In [2]:
import pandas as pd

df = pd.read_csv("reviews_preprocessed.csv")
df.head()

,summary,reviewerID,asin,reviewerName,unixReviewTime,reviewText,score,helpfulP,helpfulN
0,Disney apps,30124,2747,43149,1333843200,Disney cant make apps. They should just stick ...,1.0,6,7
1,Sucks,4713,5394,31532,1388188800,Didn't like controls and choice of weapons wou...,1.0,0,6
2,hate it,59783,7167,46230,1356048000,all it says is nice and gives you a dum!!!I wo...,1.0,2,4
3,not fun,67587,4036,41970,1334188800,not fun at all at certain times when you need ...,1.0,2,4
4,Ugh!!,65465,4625,54619,1387670400,U shud not buy this app!! Its a scam!! It do...,1.0,0,1


In [3]:
import re
import numpy as np
import nltk

stopwords = nltk.corpus.stopwords.words('english')


def get_tokens(text: str):
    tokens = nltk.word_tokenize(text)
    stems = []

    # porter = nltk.PorterStemmer()
    for token in tokens:
        if re.search("\W", token) is not None or token in stopwords:
            continue
        # stems.append(porter.stem(token))
        stems.append(token)
    return stems


def get_doc_vectors(df_data: pd.DataFrame):
    vectors_arr = np.ndarray((df_data.shape[0], embeddings.vector_size), dtype=np.float32)
    for i in df_data.index:
        tokens = get_tokens(df_data[i])
        temp = np.ndarray((len(tokens), embeddings.vector_size), dtype=np.float32)
        for j in range(len(tokens)):
            if tokens[j] in embeddings:
                temp[j] = embeddings[tokens[j]]

        temp_mean = temp.mean(axis=0)
        temp_mean = np.nan_to_num(temp_mean)
        vectors_arr[i] = temp_mean

    return vectors_arr


reviewText_vectors = get_doc_vectors(df['reviewText'])
summary_vectors = get_doc_vectors(df['summary'])
df_docs_vectors = pd.DataFrame(np.concatenate([reviewText_vectors, summary_vectors], axis=1))
df_docs_vectors.head()

C:\Users\festo\AppData\Local\Temp\ipykernel_17572\3351094019.py:30: RuntimeWarning: Mean of empty slice.
  temp_mean = temp.mean(axis=0)
c:\users\festo\onedrive - put.poznan.pl\semestr ix\emd\emd-python\venv\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,0.074479,0.032809,0.016875,0.154647,-0.130957,-0.044898,0.121012,0.053760,0.071785,-0.050234,...,0.078766,0.047852,-0.055908,-0.064941,0.042480,0.055908,-0.000977,-0.074951,0.006104,0.071777
1,0.069027,0.052856,0.085506,0.167456,-0.095322,0.076904,0.032874,-0.088348,0.039758,0.048718,...,0.273438,0.024780,-0.099609,-0.044678,0.251953,0.133789,-0.026001,-0.166992,-0.040283,-0.072266
2,0.085327,0.011286,0.024109,0.138295,-0.026159,-0.003316,0.100789,-0.043340,0.038930,0.104502,...,0.068848,0.086426,-0.061768,-0.038330,-0.174805,-0.253906,0.086914,-0.024048,-0.026978,0.125000
3,0.064137,-0.000476,-0.028126,0.157390,-0.036455,-0.046054,0.099188,0.020974,0.035567,0.046679,...,-0.018921,0.394531,-0.219727,-0.018921,-0.200195,-0.037109,0.243164,-0.135742,-0.032227,-0.072754
4,-0.035083,0.002816,0.037427,0.042627,-0.155094,0.064844,0.018351,-0.039657,0.045736,0.027482,...,0.171875,0.162109,-0.135742,0.212891,-0.052979,-0.194336,0.156250,-0.208008,0.079102,0.137695


In [4]:
# Sprawdzanie czy w danych nie ma nieleganych wartości
df_docs_vectors[df_docs_vectors.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599


In [5]:
df_result = pd.concat([df[list(set(df.columns) - {"reviewText", "summary"})], df_docs_vectors], axis=1)
df_result.columns = df_result.columns.astype(str)
df_result.to_csv("reviews_w2v.csv", index=False)